# SIDA: Speaker Identification for Archives

In [1]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio
import pandas as pd

!mkdir -p /sharedfolder/sida_classifier/

os.chdir('/sharedfolder/sida_classifier/')

training_audio_dir_name = "Applause"

In [2]:
os.chdir('/sharedfolder/sida_classifier/')

#!wget http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/females_5k.zip
#!wget http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/males_5k.zip
#!wget http://xtra.arloproject.com/datasets/aapb-ubm/Male_AAPB_171110.zip
#!wget http://xtra.arloproject.com/datasets/aapb-ubm/Female_AAPB_171110.zip

#!unzip females_5k.zip
#!unzip males_5k.zip
#!unzip Male_AAPB_171110.zip
#!unzip Female_AAPB_171110.zip

In [ ]:

#!mv females_5k Larry_Monroe/
#!mv males_5k Larry_Monroe/
#!mv Male_AAPB_171110 Larry_Monroe/
#!mv Female_AAPB_171110 Larry_Monroe/

In [ ]:
#!git clone https://github.com/hipstas/applause-classifier.git

In [ ]:
## Load 1-second labels

#csv_path = '/sharedfolder/sida_classifier/LM_random_labels_5K.csv'

#train_table_df = pd.read_csv(csv_path)

#train_table_df.head()

In [3]:
## Choosing variables to extract and assigning variables we'll use below

labels_to_use = ['Laughter', 'Non-Laughter']

media_dir_pathname = '/sharedfolder/sida_classifier/' + training_audio_dir_name

class_dir_pathname = '/sharedfolder/sida_classifier/_classes_' + training_audio_dir_name

In [4]:
## should be able to delete this cell next time 

from attk import *

import itertools

def batch_extract_vowels(media_dir):
    starting_location = os.getcwd()
    os.chdir(media_dir)
    try:
        os.mkdir('_vowel_clips')
    except:
        pass
    filenames = [item for item in os.listdir('./') if item[-4:].lower() in ('.mp3', '.wav', '.mp4')]
    for filename in filenames:
        vowel_bools = get_vowel_segments(filename, n_fft = 4096)   ##  <- edit
        vowel_ranges = labels_to_ranges(vowel_bools, label=True)
        sample_rate_val = sample_rate(filename)
        vowel_ranges_secs = []
        for pair in vowel_ranges:
            try:
                start_samples, end_samples = pair
                duration_samples = int(end_samples) - int(start_samples)
                start = float(start_samples) * (float(4096) / sample_rate_val)
                duration_secs = float(duration_samples) * (float(4096) / sample_rate_val)
                #if duration_secs >= 0.05:                                             # discarding vowel segments shorter than 0.05 sec
                vowel_ranges_secs.append((start, duration_secs))
            except Exception as e:
                print('ERROR: ' + filename)
                print(e)
        subclip_list(filename, vowel_ranges_secs, '_vowel_clips')
    os.chdir(starting_location)

    


def get_vowel_segments(media_path, n_fft=2048):
    downsample = 1
    samplerate = 44100 // downsample

    win_s = n_fft // downsample # fft size
    hop_s = n_fft  // downsample # hop size

    s = source(media_path, samplerate, hop_s)
    samplerate = s.samplerate

    tolerance = 0.6

    pitch_o = pitch("yin", win_s, hop_s, samplerate)
    pitch_o.set_unit("Hz")
    pitch_o.set_tolerance(tolerance)

    pitches = []
    confidences = []

    # total number of frames read
    total_frames = 0
    samples=[]
    pitches=[]
    while True:
        samples, read = s()
        pitch_ = pitch_o(samples)[0]
        #pitch = int(round(pitch))
        confidence = pitch_o.get_confidence()
        #print("%f %f %f" % (total_frames / float(samplerate), pitch, confidence))
        pitches += [pitch_]
        confidences += [confidence]
        total_frames += read
        if read < hop_s: break

    pitches = np.array(pitches)
    confidences = np.array(confidences)

    cleaned_pitches = ma.masked_where(confidences < tolerance, pitches)
    cleaned_pitches = ma.masked_where(cleaned_pitches > 1000, cleaned_pitches)

    try: output = list(np.logical_not(cleaned_pitches.mask))
    except: output = []

    return output

def labels_to_ranges(label_list,label=0):
    counter=0
    seq_list=[]
    for item in label_list:
        if item==label:
            seq_list.append(counter)
        counter+=1
    ranges = [(t[0][1], t[-1][1]+1) for t in (tuple(g[1]) for g in itertools.groupby(enumerate(seq_list), lambda (i, x): i - x))]
    return ranges


def sample_rate(media_path):
    return int(subprocess.check_output(['ffprobe', '-v', '0', '-of', 'csv=p=0', '-select_streams', '0', '-show_entries', 'stream=sample_rate', media_path]).strip())


In [6]:
!mkdir '/sharedfolder/sida_classifier/_classes_Applause'

In [9]:
%%capture
## Extract features (MFCCs, deltas, and delta-deltas) from all clips, then write features to CSVs

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    print("> Starting " + dir_name)
    try:
        os.chdir(os.path.join(class_dir_pathname, dir_name))
        try: os.mkdir('./_mfccs_and_deltas')
        except: pass
        filenames = [item for item in os.listdir('./') if item[-4:].lower()=='.wav']
        for filename in filenames:
            csv_out_path = './_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename, n_mfcc=30, n_fft=4096, freq_min=100, freq_max=16000)
                    if len(mfccs) > 0:
                        with open(csv_out_path, 'w') as fo:
                            csv_writer = csv.writer(fo)
                            csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        #print(e)

In [ ]:
## Continue to the next notebook to train and run the speaker ID classifier.